In [33]:
pip install s3fs

  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.8 MB/s eta 0:00:00
Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 107.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.3/272.3 kB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.7/128.7 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.1/328.1 kB 46.4 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.34.113
    Uninstalling botocore-1.34.113:
      Successfully uninstalled botocore-1.34.113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the foll

In [9]:
#from eps_utils import *
from typing import *

import numpy as np
import pandas as pd
from display import *
from call_bedrock_fast_claude_v3_py import *
import sagemaker
import boto3
from sagemaker import get_execution_role

In [10]:
pts=['WIRELESS_PLAN_OPTION',
'MOBILE_APPLICATION',
'GIFT_CARD',
'PRIME',
'CONSUMER_SERVICES_SUBSCRIPTION',
'PRESCRIPTION_DRUG',
'ABIS_EBOOKS',
'INSTALLATION_SERVICES',
'HEALTH_PERSONAL_CARE',
'BABY_PRODUCT',
'BOOKEND',
'FOOD_STORAGE_CONTAINER',
'BOTTLE',
'BED_LINEN_SET',
'DRINKING_CUP',
'TOWEL',
'INSECT_REPELLENT',
'STRING_LIGHT',
'PESTICIDE',
'LIGHT_BULB',
'ABIS_BOOK',
'MASSAGER',
'TOYS_AND_GAMES',
'TOY_FIGURE',
'HAIR_TRIMMER',
'NON_RIDING_TOY_VEHICLE',
'PUZZLES',
'TABLETOP_GAME',
'HAIR_DRYER',
'BALLOON',
'SKIN_CLEANING_AGENT',
'SKIN_MOISTURIZER',
'SHIRT',
'SHOES',
'PANTS',
'SHAMPOO',
'HAIR_STYLING_AGENT',
'WALLET',
'SUNSCREEN',
'PERSONAL_FRAGRANCE',
'CELLULAR_PHONE',
'HEADPHONES',
'CELLULAR_PHONE_CASE',
'WEARABLE_COMPUTER',
'ELECTRONIC_CABLE',
'WRITING_INSTRUMENT',
'PORTABLE_ELECTRONIC_DEVICE_COVER',
'SCREEN_PROTECTOR',
'CHARGING_ADAPTER',
'BLANK_BOOK',
'INCONTINENCE_PROTECTOR',
'VEGETABLE',
'CLEANING_AGENT',
'EDIBLE_OIL_VEGETABLE',
'FRUIT',
'HERB',
'LAUNDRY_DETERGENT',
'NUTRITIONAL_SUPPLEMENT',
'COOKIE',
'NUT_AND_SEED',
    ]


In [11]:
con=boto3.client('s3')

con.list_objects(Bucket='idq-analysis')["Contents"]
ab=! aws s3 ls --human 's3://idq-analysis/inputraw/'
sig=[i.replace('                           PRE ',"")  for i in ab if i.startswith ('                           PRE ')]
sig

['Customer_Search/', 'DPMO/', 'FAQ/']

In [51]:
prompt1='''Consider yourself as a Catalog Category Specialist.Your task is to identify valid value of an given list of attribute\
in a text.List of the attributes is provided in <att></att>.Each text separated by ;, in\
<att></att> represents individual attribute . Pick key word in <inp></inp> and tally with all attributes.Use\
 your own valid value repository and intelligence to map a valid value in the text with any attribute provided in <inp></inp>.\
Once you find a valid value\
 for an attribute you can respond in output <res></res>.Incase you get more than one attribute for the input then respond in comma separated format.\
If you dont find any attribte then respond NA.\
Make sure to output your answer for one input in <res></res>.Stop responding output of attribute. Please do not give any output apart from list provided in <inp></inp>.

<att>{a}</att>
<inp>{inp}</inp>
'''.strip()

In [52]:
sytem='Consider yourself as a Catalog Category Specialist.Your task is to identify valid value of an given list of attribute\
in a text.List of the attributes is provided in <att></att>.Each text separated by "\n" in\
<att></att> represents individual attribute . Pick key word in <inp></inp> and tally with all attributes.Use\
 your own valid value repository and intelligence to map a valid value in the text with any attribute provided in <inp></inp>.\
Once you find a valid value\
 for an attribute you can respond in output <res></res>.Incase you get more than one attribute for the input then respond in comma separated format.\
If you dont find any attribte then respond NA.\
Make sure to output your answer for one input in <res></res>.Stop responding output of attribute. Please do not give any output apart from list provided in <inp></inp>.'

prompt2='''<att>{a}</att>
<inp>{inp}</inp>
'''.strip()

In [12]:
sytem1='Consider yourself as a Catalog Category Specialist.Your task is to suggest a relevant attribute apart from the list of attributes\
by mapping the valid value which can be present in the part of the given text.List of the attributes is provided in <att></att>.Each text separated by "\n" in\
<att></att> represents individual attribute . Pick key word in <inp></inp> and tally with all attributes.Use\
 your own valid value repository and intelligence to map a valid value in the text <inp></inp> with any relevant attribute which is not available in <att></att>. You can consider list of attributes of Flipkart,Myntra from your repository to suggest a correct attribute.\
Once you find a correct attribute for the input text valid value\
 you can respond output in <res></res>.Incase you get more than one attribute for the input then respond in comma separated format.\
If you dont find any attribte then respond NA.\
Make sure to output your answer for one input in <res></res>.Stop responding output of attribute. Please do not give any output from the list provided in <att></att>.'

prompt2='''<att>{a}</att>
<inp>{inp}</inp>
'''.strip()

In [44]:
import pandas as pd
import numpy as np
def co(ll):
    ab=pd.read_parquet(ll)
    xyy=ab.groupby('product_type',as_index=False)['impressions'].describe().sort_values(by='count',ascending=False)
    xyy['std']=np.where(xyy['std'].isnull()==True,xyy['min'],xyy['std'])
    ab1=ab[ab['product_type'].isin(pts)]
    mer=pd.merge(ab1,xyy[['product_type','min']],on='product_type',how='left')
    mer=mer[mer['impressions']>=mer['min']].sort_values(['product_type','impressions'],ascending=False).drop_duplicates()
    mer=mer[~mer['keywords'].str.contains(' sale | exchange | emi | Emi | cost | Cost | Price |\
     offer | Offer | cashback | Cashback | Cash Back | cash back | COD | Return | return | cash on delivery | Cash on Delivery | Cash on delivery ')==True]
    mer['flag']=mer.groupby('product_type')['product_type'].cumcount()
    mer=mer[mer['flag']<120]
    mer['keywords'].replace(' - Genuine',"",inplace=True)
    mer.drop(columns=['min','flag'],inplace=True)
    return mer
for k in sig[1:3]:
    s3 = boto3.client('s3')
    # Replace with your bucket name and folder path
    bucket_name = 'idq-analysis' 
    prefix = 'inputraw/'+k
    result = s3.list_objects(Bucket=bucket_name,Prefix=prefix)
    ll1=['s3://idq-analysis/'+object['Key']  for object in result.get('Contents', []) if str(object['Key']).find('.parquet')>1]
    abb=pd.concat(map(co,ll1))
    df2=abb
    main=pd.DataFrame()
    for i in list(df2['product_type'].drop_duplicates()):
        df4=df2[df2['product_type']==i]
        df3=pd.read_excel(r"Copy of SF.xlsx")
        df3=df3[df3['Product Type']==i][['Product Type','V3']].drop_duplicates()
        df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
        df3.reset_index(inplace=True,drop=True)
        a="\n".join(list(df3['V3']))
        def create_root_prompt(row):
            return prompt2.format(a=a,inp=row['keywords'])
        df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
        df4.reset_index(drop=True,inplace=True)
        out=[]
        df4['Output']=call_bedrock_fast(
            prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
            model='anthropic.claude-3-sonnet-20240229-v1:0',
            system=sytem1,  ## Only works with Claude v3
            max_new_tokens=200,
            temperature=0.1,
        top_k=8,
        #top_p=1.0

            ## You can also pass: temperature, top_k, top_p, stop_sequences
        )

        df4.drop(columns='leaf_prompt',inplace=True)
        main=pd.concat([main,df4])
        main.reset_index(drop=True,inplace=True)
        main.to_parquet('output_cust_keywords2.parquet')
    from bs4 import BeautifulSoup as soup
    def ext(x,tag_name='p'):
        r=soup(x)
        c=r.find_all(tag_name)
        return str(c)
    main['attribute']=main['Output'].apply(ext,tag_name='res')
    main['attribute']=main['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)
    main['attributes']=main['attribute'].str.split(',')
    main=main.explode('attributes')
    main=main[(main['attributes']!='NA') & (main['attributes']!='[]') & (main['attributes'].isnull()!=True)]
    main['Stream']=k
    main['run_date']=pd.Timestamp.today().strftime('%Y-%m-%d')
    main[['keywords','product_type','attributes','impressions','Stream','run_date']].to_parquet('s3://idq-analysis/model_output/'+str(k).replace('/','')+'.parquet')
    main

In [42]:
master.drop_duplicates(inplace=True)
master

,keywords,product_type,attributes,impressions,Stream,run_date
0,colour pencils,WRITING_INSTRUMENT,color,617513,Customer_Search,2024-05-27
1,colour pencils,WRITING_INSTRUMENT,writing_instrument_form,617513,Customer_Search,2024-05-27
2,sketch pen set,WRITING_INSTRUMENT,brand,313387,Customer_Search,2024-05-27
3,sketch pen set,WRITING_INSTRUMENT,writing_instrument_form,313387,Customer_Search,2024-05-27
4,sketch pen set,WRITING_INSTRUMENT,subject_character,313387,Customer_Search,2024-05-27
...,...,...,...,...,...,...
342767,bibhutibhushan bandyopadhyay books,ABIS_BOOK,author,8593,Customer_Search,2024-05-30
342768,law books for students,ABIS_BOOK,subject,8461,Customer_Search,2024-05-30
342770,exam idea class 12 physics,ABIS_BOOK,exam_preparation,8196,Customer_Search,2024-05-30
342770,exam idea class 12 physics,ABIS_BOOK,academic_level,8196,Customer_Search,2024-05-30


In [ ]:
#### Main route
s3=boto3.client("s3")
bucket_name = 'idq-analysis' 
prefix = 'model_output'
result = s3.list_objects(Bucket=bucket_name,Prefix=prefix)
ll2=['s3://idq-analysis/'+object['Key']  for object in result.get('Contents', []) if str(object['Key']).find('.parquet')>1]
#abb=pd.concat(map(co,ll1))
ll2
master=pd.read_parquet('s3://idq-analysis/master-output/Master_Output.parquet')
model_output=pd.concat(map(pd.read_parquet,ll2))
model_output['Stream'].replace('\/',"",inplace=True,regex=True)
model_output
#other_output=dd.read_parquet("s3://idq-analysis/other-stream-output/*.parquet").persist().compute()
#master=pd.concat([master,model_output,other_output])
master=pd.concat([master,model_output])
master.drop_duplicates(inplace=True)
master.to_parquet('s3://idq-analysis/master-output/Master_Output.parquet')

In [66]:
ab['impressions']=ab['impressions'].replace(",","",regex=True).astype('float')
ab.groupby('product_type',as_index=False)['impressions'].describe()
#float(ab['impressions'][0].replace(",",""))


,product_type,count,mean,std,min,25%,50%,75%,max
0,3D_PRINTER,3.0,1.915667e+04,6.876130e+03,11730.0,16084.0,20438.0,22870.0,25302.0
1,ABDOMINAL_EXERCISER,5.0,1.798800e+03,1.210793e+03,36.0,1292.0,1848.0,2782.0,3036.0
2,ABIS_BOOK,7.0,6.938907e+07,7.821485e+07,19787.0,162515.5,45982198.0,121663650.0,196069174.0
3,ABIS_MUSIC,2.0,1.300900e+04,7.889897e+03,7430.0,10219.5,13009.0,15798.5,18588.0
4,ABRASIVE_SHEETS,2.0,6.019000e+03,5.825146e+03,1900.0,3959.5,6019.0,8078.5,10138.0
...,...,...,...,...,...,...,...,...,...
955,WRITING_BOARD,9.0,2.949804e+05,2.621792e+05,196.0,156841.0,288493.0,315506.0,772821.0
956,WRITING_INSTRUMENT,15.0,4.402769e+06,7.577440e+06,68.0,102545.0,1805000.0,4214825.0,27701808.0
957,WRITING_PAPER,10.0,1.512761e+05,3.056930e+05,207.0,2955.0,28989.5,92565.0,986565.0
958,YARN,1.0,1.409600e+04,NaN,14096.0,14096.0,14096.0,14096.0,14096.0


In [73]:
##other_output
ab=pd.read_parquet('Refinements data.parquet')
ab['impressions']=ab['impressions'].replace(",","",regex=True).astype('float')
xyy=ab.groupby('product_type',as_index=False)['impressions'].describe().sort_values(by='count',ascending=False)
xyy['std']=np.where(xyy['std'].isnull()==True,xyy['min'],xyy['std'])
ab1=ab[ab['product_type'].isin(pts)]
mer=pd.merge(ab1,xyy[['product_type','min']],on='product_type',how='left')
mer=mer[mer['impressions']>=mer['min']].sort_values(['product_type','impressions'],ascending=False).drop_duplicates()
mer.fillna("",inplace=True)
mer=mer[~mer['keywords'].str.contains(' sale | exchange | emi | Emi | cost | Cost | Price |\
 offer | Offer | cashback | Cashback | Cash Back | cash back | COD | Return | return | cash on delivery | Cash on Delivery | Cash on delivery ')==True]
mer['flag']=mer.groupby('product_type')['product_type'].cumcount()
mer=mer[mer['flag']<120]
mer['keywords'].replace(' - Genuine',"",inplace=True)
mer.drop(columns=['min','flag'],inplace=True)
mer['Stream']='Refinement'
mer['run_date']=pd.Timestamp.today().strftime('%Y-%m-%d')
mer.replace(["\.",".standardized_values"],['_',""],inplace=True,regex=True)
mer

/tmp/ipykernel_41792/1289407326.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  mer.fillna("",inplace=True)
/tmp/ipykernel_41792/1289407326.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mer['keywords'].replace(' - Genuine',"",inplace=True)


,keywords,product_type,attributes,impressions,Stream,run_date
468,,WRITING_INSTRUMENT,color,27701808.0,Refinement,2024-06-04
471,,WRITING_INSTRUMENT,ink_color,14468455.0,Refinement,2024-06-04
475,,WRITING_INSTRUMENT,point_type,8044473.0,Refinement,2024-06-04
472,,WRITING_INSTRUMENT,special_feature,5821932.0,Refinement,2024-06-04
473,,WRITING_INSTRUMENT,line_size,2607718.0,Refinement,2024-06-04
...,...,...,...,...,...,...
3,,ABIS_BOOK,language,110535938.0,Refinement,2024-06-04
0,,ABIS_BOOK,minimum_reading_interest_age,45982198.0,Refinement,2024-06-04
1,,ABIS_BOOK,product_group,200942.0,Refinement,2024-06-04
6,,ABIS_BOOK,breed_recommendation,124089.0,Refinement,2024-06-04


In [76]:
master1=pd.concat([master,mer])


In [27]:
##obe time ask
s3=boto3.client("s3")
bucket_name = 'idq-analysis' 
prefix = 'model_output'
result = s3.list_objects(Bucket=bucket_name,Prefix=prefix)
ll2=['s3://idq-analysis/'+object['Key']  for object in result.get('Contents', []) if str(object['Key']).find('.parquet')>1]
#abb=pd.concat(map(co,ll1))
ll2
#master=pd.read_parquet('s3://idq-analysis/master-output/Master_Output.parquet')
model_output=pd.concat(map(pd.read_parquet,ll2))
model_output['Stream'].replace('\/',"",inplace=True,regex=True)
model_output
#other_output=dd.read_parquet("s3://idq-analysis/other-stream-output/*.parquet").persist().compute()
#master=pd.concat([master,model_output,other_output])
# master=pd.concat([master,model_output])
# master.drop_duplicates(inplace=True)
# master.to_parquet('s3://idq-analysis/master-output/Master_Output.parquet')

/tmp/ipykernel_41792/200396471.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  model_output['Stream'].replace('\/',"",inplace=True,regex=True)


,keywords,product_type,attributes,impressions,Stream,run_date
0,colour pencils,WRITING_INSTRUMENT,writing_instrument_type,617513,Customer_Search,2024-05-30
1,sketch pen set,WRITING_INSTRUMENT,pen_set_type,313387,Customer_Search,2024-05-30
1,sketch pen set,WRITING_INSTRUMENT,art_supply_type,313387,Customer_Search,2024-05-30
2,white pen,WRITING_INSTRUMENT,color,125228,Customer_Search,2024-05-30
2,white pen,WRITING_INSTRUMENT,writing_instrument_form,125228,Customer_Search,2024-05-30
...,...,...,...,...,...,...
26187,"i required class 3 Organization, signature & E...",ABIS_BOOK,Product Category,1,FAQ,2024-05-30
26188,When will this reaach in india if i order it t...,ABIS_BOOK,delivery_time,1,FAQ,2024-05-30
26189,What is the warranty period for this watch ? W...,ABIS_BOOK,warranty,1,FAQ,2024-05-30
26190,Is there any service center in patna ?,ABIS_BOOK,service_center_availability,1,FAQ,2024-05-30


In [34]:
dff=model_output
df=model_output[['keywords','product_type','Stream','impressions']].drop_duplicates()
df

,keywords,product_type,Stream,impressions
0,colour pencils,WRITING_INSTRUMENT,Customer_Search,617513
1,sketch pen set,WRITING_INSTRUMENT,Customer_Search,313387
2,white pen,WRITING_INSTRUMENT,Customer_Search,125228
3,pilot fountain pen,WRITING_INSTRUMENT,Customer_Search,113194
4,sketching kit,WRITING_INSTRUMENT,Customer_Search,80034
...,...,...,...,...
26187,"i required class 3 Organization, signature & E...",ABIS_BOOK,FAQ,1
26188,When will this reaach in india if i order it t...,ABIS_BOOK,FAQ,1
26189,What is the warranty period for this watch ? W...,ABIS_BOOK,FAQ,1
26190,Is there any service center in patna ?,ABIS_BOOK,FAQ,1


In [83]:
df=master1[['keywords','product_type','Stream','impressions']].drop_duplicates()
df

,keywords,product_type,Stream,impressions
0,colour pencils,WRITING_INSTRUMENT,Customer_Search,617513.0
2,sketch pen set,WRITING_INSTRUMENT,Customer_Search,313387.0
8,white pen,WRITING_INSTRUMENT,Customer_Search,125228.0
10,pilot fountain pen,WRITING_INSTRUMENT,Customer_Search,113194.0
13,sketching kit,WRITING_INSTRUMENT,Customer_Search,80034.0
...,...,...,...,...
3,,ABIS_BOOK,Refinement,110535938.0
0,,ABIS_BOOK,Refinement,45982198.0
1,,ABIS_BOOK,Refinement,200942.0
6,,ABIS_BOOK,Refinement,124089.0


In [84]:
df.sort_values('impressions',ascending=False,inplace=True)
df['cum_impr']=df.groupby(['product_type','Stream'])['impressions'].cumsum()
df1=df.groupby(['product_type','Stream'],as_index=False)['cum_impr'].max().rename(columns={'cum_impr':'max_imr'})
df=pd.merge(df,df1,on=['product_type','Stream'],how='left')

df=pd.merge(master1,df[['keywords','product_type','Stream','cum_impr','max_imr']],on=['keywords','product_type','Stream'],how='left')
df
#df2=dff[dff['cum_impr']/df['max_imr']<=0.4]

# df2
# df1=pd.DataFrame(pd.Series(','.join(df['attribute']).split(',')).value_counts(),columns=['Counts'])

,keywords,product_type,attributes,impressions,Stream,run_date,cum_impr,max_imr
0,colour pencils,WRITING_INSTRUMENT,color,617513.0,Customer_Search,2024-05-27,13372806.0,82185683.0
1,colour pencils,WRITING_INSTRUMENT,writing_instrument_form,617513.0,Customer_Search,2024-05-27,13372806.0,82185683.0
2,sketch pen set,WRITING_INSTRUMENT,brand,313387.0,Customer_Search,2024-05-27,18973187.0,82185683.0
3,sketch pen set,WRITING_INSTRUMENT,writing_instrument_form,313387.0,Customer_Search,2024-05-27,18973187.0,82185683.0
4,sketch pen set,WRITING_INSTRUMENT,subject_character,313387.0,Customer_Search,2024-05-27,18973187.0,82185683.0
...,...,...,...,...,...,...,...,...
1368966,,ABIS_BOOK,browse_generalized_bic_geo,19787.0,Refinement,2024-06-04,439396474.0,485723490.0
1368967,,ABIS_BOOK,browse_generalized_bic_geo,19787.0,Refinement,2024-06-04,485378672.0,485723490.0
1368968,,ABIS_BOOK,browse_generalized_bic_geo,19787.0,Refinement,2024-06-04,485579614.0,485723490.0
1368969,,ABIS_BOOK,browse_generalized_bic_geo,19787.0,Refinement,2024-06-04,485703703.0,485723490.0


In [136]:
df2=df[df['cum_impr']/df['max_imr']<=0.4]

#df2.groupby(['product_type','Stream','attributes'],as_index=False)['keywords'].nunique().rename(columns={'keywords':'Key_count'}).to_csv('analysis1.csv',index=False)
df3=df2.groupby(['product_type','Stream','attributes'],as_index=False)['keywords'].nunique().rename(columns={'keywords':'Key_count'}).sort_values(by=['product_type','Stream','Key_count'],ascending=False)
df3['rank']=df3.groupby(['product_type','Stream']).cumcount()
df4=df3[df3['rank']<11]
df4['weightage']=np.where(df4['Stream']=='FAQ',0.2,np.where(df4['Stream']=='Customer_Search',0.2,np.where(df4['Stream']=='Refinement',0.2,0.4)))
#df3[['Stream','weightage']].drop_duplicates()
df4

/tmp/ipykernel_41792/3897952225.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['weightage']=np.where(df4['Stream']=='FAQ',0.2,np.where(df4['Stream']=='Customer_Search',0.2,np.where(df4['Stream']=='Refinement',0.2,0.4)))


,product_type,Stream,attributes,Key_count,rank,weightage
4712,WRITING_INSTRUMENT,FAQ,special_feature,10,0,0.2
4709,WRITING_INSTRUMENT,FAQ,ink_color,9,1,0.2
4714,WRITING_INSTRUMENT,FAQ,writing_instrument_form,9,2,0.2
4713,WRITING_INSTRUMENT,FAQ,unit_count,8,3,0.2
4708,WRITING_INSTRUMENT,FAQ,included_components,4,4,0.2
...,...,...,...,...,...,...
176,ABIS_BOOK,Customer_Search,publication_date,46,6,0.2
197,ABIS_BOOK,Customer_Search,subject,41,7,0.2
116,ABIS_BOOK,Customer_Search,author,38,8,0.2
133,ABIS_BOOK,Customer_Search,category,35,9,0.2


In [137]:
df4.sort_values(['product_type','attributes'],inplace=True)
df4['cum_weightage']=df4.groupby(['product_type','attributes'])['weightage'].cumsum()
#df4[df4['attributes']=='category']
df4=df4.groupby(['product_type','attributes'],as_index=False).agg({'Stream':list,'cum_weightage':max}).sort_values('cum_weightage',ascending=False).astype('str')
df4['Stream'].replace(['\[','\]'],"",inplace=True,regex=True)
df4.to_csv('Analysis_1st_Output.csv',index=False)

/tmp/ipykernel_41792/1777013663.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.sort_values(['product_type','attributes'],inplace=True)
/tmp/ipykernel_41792/1777013663.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['cum_weightage']=df4.groupby(['product_type','attributes'])['weightage'].cumsum()
/tmp/ipykernel_41792/1777013663.py:4: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "

In [144]:
star=pd.read_csv('Starfish.csv')
display(star)
ump=pd.read_csv('UMP.csv')
ump.fillna('UMP',inplace=True)
ump
df41=pd.merge(df4,star[['product_type','sa_name','completeness_weight']],left_on=['product_type','attributes'],right_on=['product_type','sa_name'],how='left')
df41=pd.merge(df41,ump,on=['product_type','attributes'],how='left')
df41.to_csv('Analysis_1st_Output.csv',index=False)

,Helper,product_type,sa_name,Rank,MP,completeness_weight,correctness_weight,normalization_weight,PT Scope,ASCS PT excluded,ASCS All excluded,Excluded,IDQ program PT overlap,IDQ program PT+Attribute overlap
0,3D_PRINTER-brand,3D_PRINTER,brand,1,1,8,8,8,1,0,1,1,0,0
1,3D_PRINTER-material,3D_PRINTER,material,2,1,8,8,8,1,0,0,0,0,0
2,3D_PRINTER-color,3D_PRINTER,color,3,1,8,8,8,1,0,0,0,0,0
3,3D_PRINTER-item_depth_width_height,3D_PRINTER,item_depth_width_height,4,1,4,4,4,1,0,0,0,0,0
4,3D_PRINTER-item_weight,3D_PRINTER,item_weight,5,1,4,4,4,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23415,YO_YO-eu_spare_part_availability_duration,YO_YO,eu_spare_part_availability_duration,105,1,1,1,1,1,0,0,0,0,0
23416,ZIPPER_FASTENER-color,ZIPPER_FASTENER,color,1,1,8,8,8,1,0,0,0,0,0
23417,ZIPPER_FASTENER-material,ZIPPER_FASTENER,material,2,1,4,4,4,1,0,0,0,0,0
23418,ZIPPER_FASTENER-brand,ZIPPER_FASTENER,brand,3,1,4,4,4,1,0,1,1,0,0


In [38]:
df2=dff[dff['cum_impr']/dff['max_imr']<=0.4]
df2.groupby(['product_type','Stream','attributes'],as_index=False)['keywords'].nunique().rename(columns={'keywords':'Key_count'}).to_csv('attr_out_amazon_list.csv',index=False)

,product_type,Stream,attributes,Key_count
0,ABIS_BOOK,Customer_Search,Author,2
1,ABIS_BOOK,Customer_Search,Book Type,1
2,ABIS_BOOK,Customer_Search,Credit card,1
3,ABIS_BOOK,Customer_Search,Educational Topic,1
4,ABIS_BOOK,Customer_Search,Exam Preparation Books,1
...,...,...,...,...
4583,WRITING_INSTRUMENT,FAQ,special_feature,1
4584,WRITING_INSTRUMENT,FAQ,unit_count,8
4585,WRITING_INSTRUMENT,FAQ,water_resistance,2
4586,WRITING_INSTRUMENT,FAQ,waterproof,2


In [39]:
model_output.to_csv('Attribute_list.csv')
df2.groupby(['product_type','Stream','attributes'],as_index=False)['keywords'].nunique().rename(columns={'keywords':'Key_count'}).to_csv('attr_contr_40_pct_impr.csv',index=False)

,keywords,product_type,attributes,impressions,Stream,run_date
0,colour pencils,WRITING_INSTRUMENT,writing_instrument_type,617513,Customer_Search,2024-05-30
1,sketch pen set,WRITING_INSTRUMENT,pen_set_type,313387,Customer_Search,2024-05-30
1,sketch pen set,WRITING_INSTRUMENT,art_supply_type,313387,Customer_Search,2024-05-30
2,white pen,WRITING_INSTRUMENT,color,125228,Customer_Search,2024-05-30
2,white pen,WRITING_INSTRUMENT,writing_instrument_form,125228,Customer_Search,2024-05-30
...,...,...,...,...,...,...
26187,"i required class 3 Organization, signature & E...",ABIS_BOOK,Product Category,1,FAQ,2024-05-30
26188,When will this reaach in india if i order it t...,ABIS_BOOK,delivery_time,1,FAQ,2024-05-30
26189,What is the warranty period for this watch ? W...,ABIS_BOOK,warranty,1,FAQ,2024-05-30
26190,Is there any service center in patna ?,ABIS_BOOK,service_center_availability,1,FAQ,2024-05-30


In [ ]:
main=pd.DataFrame()
for i in list(df2['product_type'].drop_duplicates()):
    df4=df2[df2['product_type']==i]
    df3=pd.read_excel(r"Copy of SF.xlsx")
    df3=df3[df3['Product Type']==i][['Product Type','V3']].drop_duplicates()
    df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
    df3.reset_index(inplace=True,drop=True)
    a="\n".join(list(df3['V3']))
    def create_root_prompt(row):
        return prompt1.format(a=a,inp=row['keywords'])
    df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
    df4.reset_index(drop=True,inplace=True)
    out=[]
    df4['Output']=call_bedrock_fast(
        prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
        model='anthropic.claude-3-haiku-20240307-v1:0',
        system=sytem,  ## Only works with Claude v3
        max_new_tokens=200,
        temperature=0.1,
    top_k=5,
    #top_p=1.0
    
        ## You can also pass: temperature, top_k, top_p, stop_sequences
    )
       
    df4.drop(columns='leaf_prompt',inplace=True)
    main=pd.concat([main,df4])
    main.reset_index(drop=True,inplace=True)
    main.to_parquet('output_cust_keywords2.parquet')

In [ ]:
main=pd.read_parquet('output_cust_keywords2.parquet')
from bs4 import BeautifulSoup as soup
def ext(x,tag_name='p'):
    r=soup(x)
    c=r.find_all(tag_name)
    return str(c)
main['attribute']=main['Output'].apply(ext,tag_name='res')
main['attribute']=main['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)
main

In [ ]:
main['attributes']=main['attribute'].str.split(',')
main=main.explode('attributes')
main=main[(main['attributes']!='NA') & (main['attributes']!='[]') & (main['attributes'].isnull()!=True)]

In [ ]:

main['Stream']=i
main['run_date']=pd.Timestamp.today().strftime('%Y-%m-%d')
main[['keywords','product_type','attributes','Stream','run_date']].to_parquet('s3://idq-analysis/model_output/'+str(i)+'.parquet')
main

In [ ]:
master=pd.read_parquet('s3://idq-analysis/master-output/Master_Output.parquet')
model_output=dd.read_parquet("s3://idq-analysis/model_output/*.parquet").persist().compute()
other_output=dd.read_parquet("s3://idq-analysis/other-stream-output/*.parquet").persist().compute()
master=pd.concat([master,model_output,other_output])
master.drop_duplicates(inplace=True)
master.to_parquet('s3://idq-analysis/master-output/Master_Output.parquet')

In [59]:
pd.read_parquet('s3://idq-analysis/model_output/INSTALLATION_SERVICES.parquet')

,keywords,product_type,attributes,impressions,Stream,run_date
0,How many pens?,WRITING_INSTRUMENT,unit_count,55,FAQ/,2024-05-28
1,Is it a gel pen,WRITING_INSTRUMENT,writing_instrument_form,41,FAQ/,2024-05-28
2,Ink colour,WRITING_INSTRUMENT,ink_color,41,FAQ/,2024-05-28
3,Is it a gel pen?,WRITING_INSTRUMENT,writing_instrument_form,28,FAQ/,2024-05-28
4,Is it gel pen?,WRITING_INSTRUMENT,writing_instrument_form,28,FAQ/,2024-05-28
...,...,...,...,...,...,...
26175,B P can Messer,ABIS_BOOK,contributor,1,FAQ/,2024-05-28
26181,I need to print about 200 pages per month....i...,ABIS_BOOK,pages,1,FAQ/,2024-05-28
26181,I need to print about 200 pages per month....i...,ABIS_BOOK,item_weight,1,FAQ/,2024-05-28
26182,How many pages does the book has,ABIS_BOOK,pages,1,FAQ/,2024-05-28


In [4]:

df2=pd.read_excel('Copy of SF.xlsx')
df3=df2[['Product Type','V3']].drop_duplicates()
df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
df3


/tmp/ipykernel_32355/2986301580.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['V3'].replace(["\."],["_"],inplace=True,regex=True)


,Product Type,V3
0,WATER_DISPENSER,access_location
1,MINERAL_SUPPLEMENT,active_ingredient_strength
2,PROTEIN_SUPPLEMENT_POWDER,active_ingredient_strength
3,ABIS_BOOK,language
4,ESSENTIAL_OIL,active_ingredients
...,...,...
7651,PHONE,refresh_rate
7652,PHONE,resolution
7653,PHONE,sim_card_slot_count
7654,PHONE,warranty_description


In [42]:
#print(df2.iloc[0,2])
cus=pd.read_parquet("customer_search_records.parquet")
main_ou=pd.read_parquet("s3://idq-analysis/master-output/Master_Output.parquet")
#pd.merge(main_ou,cus[['keywords','product_type','impressions']],on=['keywords','product_type'],how='left')[['keywords', 'product_type', 'attributes','impressions','Stream', 'run_date',]].to_parquet("s3://idq-analysis/master-output/Master_Output.parquet")
main_ou.to_parquet("s3://idq-analysis/model_output/Customer_Search.parquet")

In [5]:
import pandas as pd
#del df2
df2=pd.read_parquet("customer_search_records.parquet")[['keywords','product_type']]

df3=pd.read_excel(r"Copy of SF.xlsx")
df3=df3[['V3','is prod fact attribute']].drop_duplicates()
df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
#df3['V3']=df3['V3'].apply(lambda x: "<'item'>"+x+"<'/item'>")
df3.reset_index(inplace=True,drop=True)
a="\n".join(list(df3['V3']))
def create_root_prompt(row):
    return prompt2.format(a=a,inp=row['keywords'])
df2['leaf_prompt']=df2.apply(create_root_prompt,axis=1)
df2.reset_index(drop=True,inplace=True)
df2

/tmp/ipykernel_32355/3965612381.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['V3'].replace(["\."],["_"],inplace=True,regex=True)


,keywords,product_type,leaf_prompt
0,colour pencils,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...
1,sketch pen set,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...
2,white pen,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...
3,pilot fountain pen,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...
4,sketching kit,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...
...,...,...,...
340818,bibhutibhushan bandyopadhyay books,ABIS_BOOK,<att>access_location\nactive_ingredient_streng...
340819,law books for students,ABIS_BOOK,<att>access_location\nactive_ingredient_streng...
340820,ettavum priyappetta ennodu,ABIS_BOOK,<att>access_location\nactive_ingredient_streng...
340821,exam idea class 12 physics,ABIS_BOOK,<att>access_location\nactive_ingredient_streng...


In [ ]:
print(df2.iloc[0,2])

In [ ]:
call_bedrock_fast(
        prompts=['''Human:
        hello Assistant:'''],  ## REPLACE THIS WITH YOUR PROMPT!!
        model='anthropic.claude-v2',
        system='You are an expert at Catalog',  ## Only works with Claude v3
        max_new_tokens=200,
        temperature=0.1
        ## You can also pass: temperature, top_k, top_p, stop_sequences
    )

In [14]:
d3=df2.iloc[0:15,:]
d3['ans']=call_bedrock_fast(
        prompts=d3['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
        model='anthropic.claude-3-haiku-20240307-v1:0',
        system=sytem,  ## Only works with Claude v3
        max_new_tokens=200,
        temperature=0.1,
    top_k=5,
    #top_p=1.0
    
        ## You can also pass: temperature, top_k, top_p, stop_sequences
    )
d3

100%|███████████████████████████████████████████████████████████████| 15/15 [00:02<00:00,  5.77it/s]
/tmp/ipykernel_32355/2683474136.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d3['ans']=call_bedrock_fast(


,keywords,product_type,leaf_prompt,ans
0,colour pencils,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,<res>color</res>
1,sketch pen set,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,"<res>item_length_width_height, item_weight, nu..."
2,white pen,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,<res>writing_instrument_form</res>
3,pilot fountain pen,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,"<res>item_form, material, writing_instrument_f..."
4,sketching kit,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,"<res>art_medium, art_supplies</res>"
5,classmate octane ball pen,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,"<res>item_form,brand,color,style</res>"
6,4 in 1 pen,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,<res>writing_instrument_form</res>
7,customised pen with name,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,"<res>writing_instrument_form, customization</res>"
8,drawing pencils set,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,"<res>item_length_width_height, item_weight, nu..."
9,gallery oil pastels,WRITING_INSTRUMENT,<att>access_location\nactive_ingredient_streng...,<res>art_medium</res>


In [14]:
main=pd.DataFrame()
for i in list(df2['product_type'].drop_duplicates()):
    df4=df2[df2['product_type']==i]
    df3=pd.read_excel(r"Copy of SF.xlsx")
    df3=df3[df3['Product Type']==i][['Product Type','V3']].drop_duplicates()
    df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
    df3.reset_index(inplace=True,drop=True)
    a="\n".join(list(df3['V3']))
    def create_root_prompt(row):
        return prompt1.format(a=a,inp=row['keywords'])
    df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
    df4.reset_index(drop=True,inplace=True)
    out=[]
    df4['Output']=call_bedrock_fast(
        prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
        model='anthropic.claude-3-haiku-20240307-v1:0',
        system=sytem,  ## Only works with Claude v3
        max_new_tokens=200,
        temperature=0.1,
    top_k=5,
    #top_p=1.0
    
        ## You can also pass: temperature, top_k, top_p, stop_sequences
    )
       
    df4.drop(columns='leaf_prompt',inplace=True)
    main=pd.concat([main,df4])
    main.reset_index(drop=True,inplace=True)
    main.to_parquet('output_cust_keywords2.parquet')

In [75]:

main=pd.read_parquet('output_cust_keywords2.parquet')
from bs4 import BeautifulSoup as soup
def ext(x,tag_name='p'):
    r=soup(x)
    c=r.find_all(tag_name)
    return str(c)
main['attribute']=main['Output'].apply(ext,tag_name='res')
main['attribute']=main['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)
main

/tmp/ipykernel_19150/3268480615.py:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  r=soup(x)


,keywords,product_type,Output,attribute
0,colour pencils,WRITING_INSTRUMENT,"<res>color, writing_instrument_form</res>","color, writing_instrument_form"
1,sketch pen set,WRITING_INSTRUMENT,"<res>brand,writing_instrument_form,subject_cha...","brand,writing_instrument_form,subject_characte..."
2,white pen,WRITING_INSTRUMENT,"<res>color,writing_instrument_form</res>","color,writing_instrument_form"
3,pilot fountain pen,WRITING_INSTRUMENT,"<res>brand,writing_instrument_form,ink_color</...","brand,writing_instrument_form,ink_color"
4,sketching kit,WRITING_INSTRUMENT,"<res>brand,material,recommended_uses_for_produ...","brand,material,recommended_uses_for_product"
...,...,...,...,...
340818,bibhutibhushan bandyopadhyay books,ABIS_BOOK,"<res>language, contributor, genre, series_titl...","language, contributor, genre, series_title"
340819,law books for students,ABIS_BOOK,"<res>genre, item_dimensions, pages</res>","genre, item_dimensions, pages"
340820,ettavum priyappetta ennodu,ABIS_BOOK,<res>NA</res>,NA
340821,exam idea class 12 physics,ABIS_BOOK,NA,[]


In [77]:
main['attributes']=main['attribute'].str.split(',')
main=main.explode('attributes')
main=main[(main['attributes']!='NA') & (main['attributes']!='[]') & (main['attributes'].isnull()!=True)]
#main
# def aggr(x):
#     name={
#         'Total_Count_attributes': x['exp_att'].count(),
#         # 'Leaf_node=Y':x[x['leaf_validate']==True]['asin'].drop_duplicates().count(),
#         # 'sum_Gvs':pd.to_numeric(x[['asin','category','Total_Gvs']].drop_duplicates()['Total_Gvs']).sum(),
#         # 'total_orders':pd.to_numeric(x[['asin','category','total_ordered_units']].drop_duplicates()['total_ordered_units']).sum(),
        
#     }
#     return pd.Series(name)
# main.groupby(['product_type','exp_att'],as_index=False)['keywords'].nunique()
#agg( {'exp_att':drop_duplicates()})


In [79]:
k='Customer_Search'
main['Stream']=k
main['run_date']=pd.Timestamp.today().strftime('%Y-%m-%d')
main[['keywords','product_type','attributes','Stream','run_date']].to_parquet('s3://idq-analysis/model_output/'+str(k)+'.parquet')
main

,keywords,product_type,Output,attribute,attributes,Stream,run_date
0,colour pencils,WRITING_INSTRUMENT,"<res>color, writing_instrument_form</res>","color, writing_instrument_form",color,Customer_Search,2024-05-27
0,colour pencils,WRITING_INSTRUMENT,"<res>color, writing_instrument_form</res>","color, writing_instrument_form",writing_instrument_form,Customer_Search,2024-05-27
1,sketch pen set,WRITING_INSTRUMENT,"<res>brand,writing_instrument_form,subject_cha...","brand,writing_instrument_form,subject_characte...",brand,Customer_Search,2024-05-27
1,sketch pen set,WRITING_INSTRUMENT,"<res>brand,writing_instrument_form,subject_cha...","brand,writing_instrument_form,subject_characte...",writing_instrument_form,Customer_Search,2024-05-27
1,sketch pen set,WRITING_INSTRUMENT,"<res>brand,writing_instrument_form,subject_cha...","brand,writing_instrument_form,subject_characte...",subject_character,Customer_Search,2024-05-27
...,...,...,...,...,...,...,...
340819,law books for students,ABIS_BOOK,"<res>genre, item_dimensions, pages</res>","genre, item_dimensions, pages",genre,Customer_Search,2024-05-27
340819,law books for students,ABIS_BOOK,"<res>genre, item_dimensions, pages</res>","genre, item_dimensions, pages",item_dimensions,Customer_Search,2024-05-27
340819,law books for students,ABIS_BOOK,"<res>genre, item_dimensions, pages</res>","genre, item_dimensions, pages",pages,Customer_Search,2024-05-27
340822,meera sadhu book,ABIS_BOOK,"<res>genre, series_title</res>","genre, series_title",genre,Customer_Search,2024-05-27


In [18]:
#! aws s3 sync s3://cradle-dumps-2/sbiswaji/keywords/2024-05-02/ s3://idq-analysis/inputraw/ 

In [31]:
import sagemaker
import boto3
from sagemaker import get_execution_role

In [66]:
con=boto3.client('s3')

#con.list_objects(Bucket='idq-analysis')["Contents"]
ab=! aws s3 ls --human 's3://idq-analysis/inputraw/'
[i.replace('                           PRE ',"")  for i in ab if i.startswith ('                           PRE ')]

['Customer_Search/', 'DPMO/', 'FAQ/']

In [71]:
path=r"C:\Users\sbiswaji\Documents\test_json.csv"
import os
os.path.dirname('s3://idq-analysis/inputraw/Customer_Search/')
os.path.splitext(os.path.basename(path))
os.path.basename(path)
pd.Timestamp.today().strftime('%Y-%m-%d')

'2024-05-27'

In [ ]:
import ray, time, glob,os
import dask.dataframe as dd
from ray.util.dask import ray_dask_get, enable_dask_on_ray, disable_dask_on_ray
import boto3

In [ ]:
master=pd.read_parquet('s3://idq-analysis/master-output/Master_Output.parquet')
model_output=dd.read_parquet("s3://idq-analysis/model_output/*.parquet").persist().compute()
other_output=dd.read_parquet("s3://idq-analysis/other-stream-output/*.parquet").persist().compute()
master=pd.concat([master,model_output,other_output])
master.drop_duplicates(inplace=True)
master.to_parquet('s3://idq-analysis/master-output/Master_Output.parquet')

Analysis

In [ ]:
main

In [103]:
av=pd.DataFrame(main['attributes'].value_counts())
av[av['count']>100]

,count
attributes,
brand,153656
material,28677
item_form,25438
special_feature,17047
target_gender,16211
...,...
educational_objective,109
battery_capacity,106
account,105


In [7]:
out=[]
main=pd.DataFrame()
for generated_text in call_bedrock_fast(
    prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
    model='anthropic.claude-3-sonnet-20240229-v1:0',
    system='You are an expert at Catalog',  ## Only works with Claude v3
    max_new_tokens=200,
    temperature=0.1
    ## You can also pass: temperature, top_k, top_p, stop_sequences
):
    out.append(generated_text)
df4.drop(columns='leaf_prompt',inplace=True)
df4['Output']=out

main=pd.concat([main,df4])
main.reset_index(drop=True,inplace=True)
main.to_parquet('output_keywords.parquet')

100%|███████████████████████████████████████████████████████| 20001/20001 [2:12:53<00:00,  2.51it/s]
/tmp/ipykernel_9381/1134079663.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop(columns='leaf_prompt',inplace=True)
/tmp/ipykernel_9381/1134079663.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Output']=out


In [36]:
df2.loc[40001:len(df2),:]

,keywords,product_type
40001,calorie free salad dressing,SALAD_DRESSING
40002,salad spinner and dryer,SALAD_DRESSING
40003,pizza pasta sauce,SALAD_DRESSING
40004,boombay,SALAD_DRESSING
40005,mozzarella cheese 1kg,SALAD_DRESSING
...,...,...
151301,3d chashma,3D_GLASSES
151302,3d passive glasses,3D_GLASSES
151303,blue and red glass,3D_GLASSES
151304,air vision pro,3D_GLASSES


/tmp/ipykernel_6828/1466111506.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df3['V3'].replace(["\."],["_"],inplace=True,regex=True)


NameError: name 'prompt1' is not defined

In [35]:
out=[]

for generated_text in call_bedrock_fast(
    prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
    model='anthropic.claude-3-sonnet-20240229-v1:0',
    system='You are an expert at Catalog',  ## Only works with Claude v3
    max_new_tokens=200,
    temperature=0.1
    ## You can also pass: temperature, top_k, top_p, stop_sequences
):
    out.append(generated_text)
df4.drop(columns='leaf_prompt',inplace=True)
df4['Output']=out




100%|███████████████████████████████████████████████████████| 31305/31305 [3:42:09<00:00,  2.35it/s]
/tmp/ipykernel_9381/2006704648.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop(columns='leaf_prompt',inplace=True)
/tmp/ipykernel_9381/2006704648.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Output']=out


In [36]:
df4

,keywords,product_type,Output
0,robux,CONSOLE_VIDEO_GAMES,<in>robux</in>\n<res>NA</res>
1,xbox 360,CONSOLE_VIDEO_GAMES,<in>xbox 360</in>\n<res>NA</res>
2,gta 5,CONSOLE_VIDEO_GAMES,<in>gta 5</in>\n<res>NA</res>
3,xbox game pass ultimate,CONSOLE_VIDEO_GAMES,<in>xbox game pass ultimate</in>\n<res>NA</res>
4,wwe 2k24,CONSOLE_VIDEO_GAMES,<in>wwe 2k24</in>\n<res>NA</res>
...,...,...,...
31300,3d chashma,3D_GLASSES,<in>3d chashma</in>\n<res>NA</res>
31301,3d passive glasses,3D_GLASSES,<in>3d passive glasses</in>\n<res>item_form</res>
31302,blue and red glass,3D_GLASSES,<in>blue and red glass</in>\n<res>color</res>
31303,air vision pro,3D_GLASSES,<in>air vision pro</in>\n<res>NA</res>


In [37]:
main=pd.concat([main,df4],ignore_index=True)
main.reset_index(drop=True,inplace=True)
main.to_parquet('output_keywords.parquet')
main

,keywords,product_type,Output,attribute
0,zipper,ZIPPER_FASTENER,<in>zipper</in>\n<res>closure_type</res>,closure_type
1,detachable zipper puller,ZIPPER_FASTENER,<in>detachable zipper puller</in>\n<res>closur...,closure_material
2,zip,ZIPPER_FASTENER,<in>zip</in>\n<res>NA</res>,NA
3,invisible cloth,ZIPPER_FASTENER,<in>invisible cloth</in>\n<res>material_type_f...,material_type_free
4,zipper puller,ZIPPER_FASTENER,<in>zipper puller</in>\n<res>closure_material<...,closure_material
...,...,...,...,...
151301,3d chashma,3D_GLASSES,<in>3d chashma</in>\n<res>NA</res>,NaN
151302,3d passive glasses,3D_GLASSES,<in>3d passive glasses</in>\n<res>item_form</res>,NaN
151303,blue and red glass,3D_GLASSES,<in>blue and red glass</in>\n<res>color</res>,NaN
151304,air vision pro,3D_GLASSES,<in>air vision pro</in>\n<res>NA</res>,NaN


In [2]:
from bs4 import BeautifulSoup as soup
def ext(x,tag_name='p'):
    r=soup(x)
    c=r.find_all(tag_name)
    return str(c)
main['attribute']=main['Output'].apply(ext,tag_name='res')
main['attribute']=main['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)
main

NameError: name 'main' is not defined

In [39]:
from bs4 import BeautifulSoup as soup
def ext(x,tag_name='p'):
    r=soup(x)
    c=r.find_all(tag_name)
    return str(c)
df4['attribute']=df4['Output'].apply(ext,tag_name='res')
df4['attribute']=df4['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)
df4

/tmp/ipykernel_9381/2489985423.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['attribute']=df4['Output'].apply(ext,tag_name='res')
/tmp/ipykernel_9381/2489985423.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['attribute']=df4['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)


,keywords,product_type,Output,attribute
0,robux,CONSOLE_VIDEO_GAMES,<in>robux</in>\n<res>NA</res>,NA
1,xbox 360,CONSOLE_VIDEO_GAMES,<in>xbox 360</in>\n<res>NA</res>,NA
2,gta 5,CONSOLE_VIDEO_GAMES,<in>gta 5</in>\n<res>NA</res>,NA
3,xbox game pass ultimate,CONSOLE_VIDEO_GAMES,<in>xbox game pass ultimate</in>\n<res>NA</res>,NA
4,wwe 2k24,CONSOLE_VIDEO_GAMES,<in>wwe 2k24</in>\n<res>NA</res>,NA
...,...,...,...,...
31300,3d chashma,3D_GLASSES,<in>3d chashma</in>\n<res>NA</res>,NA
31301,3d passive glasses,3D_GLASSES,<in>3d passive glasses</in>\n<res>item_form</res>,item_form
31302,blue and red glass,3D_GLASSES,<in>blue and red glass</in>\n<res>color</res>,color
31303,air vision pro,3D_GLASSES,<in>air vision pro</in>\n<res>NA</res>,NA


In [40]:
df2=pd.read_parquet('keywords_top_100.parquet')
df2
df5=pd.merge(df4,df2,on=['keywords','product_type'])
df5.drop(columns='Output',inplace=True)
df5.to_csv('keywords_30K_attributes.csv',index=False)

In [24]:
prompt1='''Consider yourself as a Catalog Category Specialist.Your task is to identify valid value of an given list of attribute\
in a text.List of the attributes is provided in <att></att>.Each text separated by \n in\
<att></att> represents individual attribute . Pick key word in <inp></inp> and tally with all attributes and use\
your own valid value repository, intelligence to check if a valid value corresponding to any of the attribute is present in the text.\
Once you find a valid value\
for an attribute you can respond output in <res></res>.Incase you get more than one attribute for the input then respond in comma separated format.\
If you dont find any attribte then respond NA.\
Put input provided in <in></in>.Make sure to output your answer for one input in <res></res>.Stop responding after output of attribute.

<att>{a}</att>
<inp>{inp}</inp>
'''

In [7]:
ky=pd.read_csv('Input file Return.csv')
ky1=ky[['product_type.value','Keywords']].drop_duplicates()
ky1['Keywords'].replace(' - Genuine',"",inplace=True,regex=True)
ky1.reset_index(drop=True,inplace=True)
gr=ky1.groupby('product_type.value')

In [26]:
ky=pd.read_parquet('QnA_top_5ASINs.parquet')
ky1=ky[['product_type','question_text']].drop_duplicates()
#ky1['Keywords'].replace(' - Genuine',"",inplace=True,regex=True)
ky1.reset_index(drop=True,inplace=True)
ky1
#gr=ky1.groupby('product_type.value')

,product_type,question_text
0,SHIRT,Size
1,HEADPHONES,Warranty
2,SHIRT,Size chart
3,CELLULAR_PHONE,Exchange offer
4,CELLULAR_PHONE,Exchange
...,...,...
42803,CELLULAR_PHONE,How about camera?
42804,WEARABLE_COMPUTER,Does it have Play store
42805,CELLULAR_PHONE,is this phone good?
42806,CELLULAR_PHONE,How is the phone?


In [34]:
main=pd.DataFrame()
for i in list(ky1['product_type'].drop_duplicates()):
    df4=ky1[ky1['product_type']==i]
    df3=pd.read_excel(r"Copy of SF.xlsx")
    df3=df3[df3['Product Type']==i][['Product Type','V3']].drop_duplicates()
    df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
    df3.reset_index(inplace=True,drop=True)
    a="\n".join(list(df3['V3']))
    def create_root_prompt(row):
        return prompt1.format(a=a,inp=row['question_text'])
    df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
    df4.reset_index(drop=True,inplace=True)
    out=[]
    for generated_text in call_bedrock_fast(
        prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
        model='anthropic.claude-3-sonnet-20240229-v1:0',
        system='You are an expert at Catalog',  ## Only works with Claude v3
        max_new_tokens=200,
        temperature=0.1
        ## You can also pass: temperature, top_k, top_p, stop_sequences
    ):
        out.append(generated_text)
    df4.drop(columns='leaf_prompt',inplace=True)
    df4['Output']=out
    main=pd.concat([main,df4])
    main.reset_index(drop=True,inplace=True)
    main.to_parquet('output_keywords.parquet')

/tmp/ipykernel_79375/4188437185.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
100%|███████████████████████████████████████████████████████████| 3890/3890 [06:25<00:00, 10.10it/s]
/tmp/ipykernel_79375/4188437185.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4.drop(columns='leaf_prompt',inplace=True)
/tmp/ipykernel_79375/4188437185.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [36]:
main

,product_type,question_text,Output
0,SHIRT,Size,<res>shirt_size</res>
1,SHIRT,Size chart,<in>Size chart</in>\n<res>shirt_size</res>
2,SHIRT,Size?,<in>Size?</in>\n<res>shirt_size</res>
3,SHIRT,Size chart?,<in>Size chart?</in>\n<res>shirt_size</res>
4,SHIRT,What is the size?,<in>What is the size?</in>\n<res>shirt_size</res>
...,...,...,...
42803,ABIS_BOOK,Is this a real dog,<in>Is this a real dog</in>\n<res>NA</res>
42804,ABIS_BOOK,It is real dog,<in>It is real dog</in>\n<res>NA</res>
42805,ABIS_BOOK,How many pages in this book?,<in>How many pages in this book?</in>\n<res>pa...
42806,ABIS_BOOK,How many pages in this book,<in>How many pages in this book</in>\n<res>pag...


In [35]:
# main=pd.DataFrame()
# for i in list(ky1['product_type'].drop_duplicates()):
#     df4=ky1[ky1['product_type']==i]
#     df3=pd.read_excel(r"Copy of SF.xlsx")
#     df3=df3[df3['Product Type']==i][['Product Type','V3']].drop_duplicates()
#     df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
#     df3.reset_index(inplace=True,drop=True)
#     a="\n".join(list(df3['V3']))
#     def create_root_prompt(row):
#         return prompt1.format(a=a,inp=row['Keywords'])
#     df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
#     df4.reset_index(drop=True,inplace=True)
#     out=[]
#     for generated_text in call_bedrock_fast(
#         prompts=df4['leaf_prompt'],  ## REPLACE THIS WITH YOUR PROMPT!!
#         model='anthropic.claude-3-sonnet-20240229-v1:0',
#         system='You are an expert at Catalog',  ## Only works with Claude v3
#         max_new_tokens=200,
#         temperature=0.1
#         ## You can also pass: temperature, top_k, top_p, stop_sequences
#     ):
#         out.append(generated_text)
#     df4.drop(columns='leaf_prompt',inplace=True)
#     df4['Output']=out
#     main=pd.concat([main,df4])
#     main.reset_index(drop=True,inplace=True)
#     main.to_parquet('output_keywords.parquet')

In [ ]:
df4=df2.loc[0:20000,:]
df3=pd.read_excel(r"Copy of SF.xlsx")
df3=df3[['V3','is prod fact attribute']].drop_duplicates()
df3['V3'].replace(["\."],["_"],inplace=True,regex=True)
#df3['V3']=df3['V3'].apply(lambda x: "<'item'>"+x+"<'/item'>")
df3.reset_index(inplace=True,drop=True)
a="\n".join(list(df3['V3']))
def create_root_prompt(row):
    return prompt1.format(a=a,inp=row['keywords'])
df4['leaf_prompt']=df4.apply(create_root_prompt,axis=1)
df4.reset_index(drop=True,inplace=True)
df4

In [13]:
pd.read_parquet('output_keywords.parquet')

,product_type.value,Keywords,Output
0,ABDOMINAL_EXERCISER,Call Issue,<in>Call Issue</in>\n<res>NA</res>
1,ABDOMINAL_EXERCISER,Calls issue,<in>Calls issue</in>\n<res>NA</res>
2,ABDOMINAL_EXERCISER,DOA/Stopped Working-Genuine,<in>DOA/Stopped Working-Genuine</in>\n<res>NA<...
3,ABIS_BOOK,Print Issue,<in>Print Issue</in>\n<res>binding</res>
4,ABIS_BOOK,Used Item,<in>Used Item</in>\n<res>NA</res>
...,...,...,...
4611,WRITING_INSTRUMENT,Other Defective Issue,<in>Other Defective Issue</in>\n<res>NA</res>
4612,WRITING_INSTRUMENT,DOA/Stopped working- Genuine,<in>DOA/Stopped working- Genuine</in>\n<res>NA...
4613,WRITING_INSTRUMENT,Scroll Wheel Issue,<in>Scroll Wheel Issue</in>\n<res>NA</res>
4614,WRITING_INSTRUMENT,Elastic Issue,<in>Elastic Issue</in>\n<res>NA</res>


In [37]:
from bs4 import BeautifulSoup as soup
def ext(x,tag_name='p'):
    r=soup(x)
    c=r.find_all(tag_name)
    return str(c)
main['attribute']=main['Output'].apply(ext,tag_name='res')
main['attribute']=main['attribute'].astype('str').replace(['\[<res>','</res>\]'],"",regex=True)
main

,product_type,question_text,Output,attribute
0,SHIRT,Size,<res>shirt_size</res>,shirt_size
1,SHIRT,Size chart,<in>Size chart</in>\n<res>shirt_size</res>,shirt_size
2,SHIRT,Size?,<in>Size?</in>\n<res>shirt_size</res>,shirt_size
3,SHIRT,Size chart?,<in>Size chart?</in>\n<res>shirt_size</res>,shirt_size
4,SHIRT,What is the size?,<in>What is the size?</in>\n<res>shirt_size</res>,shirt_size
...,...,...,...,...
42803,ABIS_BOOK,Is this a real dog,<in>Is this a real dog</in>\n<res>NA</res>,NA
42804,ABIS_BOOK,It is real dog,<in>It is real dog</in>\n<res>NA</res>,NA
42805,ABIS_BOOK,How many pages in this book?,<in>How many pages in this book?</in>\n<res>pa...,pages
42806,ABIS_BOOK,How many pages in this book,<in>How many pages in this book</in>\n<res>pag...,pages


In [23]:
ky['Keywords'].replace(' - Genuine',"",regex=True,inplace=True)
ky

,Asin,product_type.value,Keywords
0,B08K7QZK8Q,SLIPPER,Used Item
1,B09ZJ1HR4B,BOTTLE,Poor Quality Issue
2,B0CKWTJ6LY,BOTTLE,Does not meet expectation/Requirement
3,B0CBBNQ2ZZ,SHOES,Used Item
4,B0CQNT6576,BED_LINEN_SET,Used Item
...,...,...,...
110077,B0BCZQLRM8,SHIRT,Clothing Attribute Issue
110078,B08QRZQDKJ,SHIRT,Stain Issue
110079,1847942792,ABIS_BOOK,Fake Product
110080,B0CJ6WQNWP,CELLULAR_PHONE_CASE,Used Item


In [42]:
ky.reset_index(inplace=True,drop=True)
pd.merge(ky,main,on=['product_type','question_text'],how='inner').drop(columns='Output').to_csv('QnA_output.csv',index=False)

In [47]:
! aws s3 cp s3://saursunndrag/IDQ_SearchData/Parquet-Format/HPC_DASH_20240504000 s3://idq-cradle-output-bucket/HPC_Dashboard/HPC_DASH_20240504000

copy failed: s3://saursunndrag/IDQ_SearchData/Parquet-Format/HPC_DASH_20240504000 to s3://idq-cradle-output-bucket/HPC_Dashboard/HPC_DASH_20240504000 An error occurred (AccessDenied) when calling the CreateMultipartUpload operation: Access Denied
